In [1]:
# @title Coleta de Dados de Contratações PNCP

!pip install pandas requests
import requests
import pandas as pd
from datetime import datetime, timedelta
import calendar

# Função para coletar dados da API com ajuste automático do tamanho da página
def coletar_dados(codigo_unidade, data_publicacao_inicial, data_publicacao_final, codigo_modalidade, pagina=1, tamanho_pagina=10):
    url = f"https://dadosabertos.compras.gov.br/modulo-contratacoes/1_consultarContratacoes_PNCP_14133"

    while tamanho_pagina > 0:  # Continua tentando até encontrar um tamanho de página aceitável
        params = {
            'pagina': pagina,
            'tamanhoPagina': tamanho_pagina,
            'unidadeOrgaoCodigoUnidade': codigo_unidade,
            'dataPublicacaoPncpInicial': data_publicacao_inicial,
            'dataPublicacaoPncpFinal': data_publicacao_final,
            'codigoModalidade': codigo_modalidade
        }
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()  # Retorna os dados quando bem-sucedido
        else:
            print(f"Erro ao coletar dados (status {response.status_code}) com tamanho da página {tamanho_pagina}. Tentando tamanho menor...")
            tamanho_pagina -= 10  # Reduz o tamanho da página em 10 a cada tentativa

    print("Não foi possível coletar os dados. Tente ajustar os parâmetros.")
    return None

# Função para salvar os dados em uma planilha
def salvar_planilha(dados, nome_arquivo):
    df = pd.json_normalize(dados['resultado'])
    df.to_excel(nome_arquivo, index=False)
    print(f"Planilha salva como {nome_arquivo}")

# Função para gerar intervalos mensais
def gerar_intervalos(data_inicial, data_final):
    intervalos = []
    while data_inicial <= data_final:
        # Define o último dia do mês atual
        ultimo_dia = calendar.monthrange(data_inicial.year, data_inicial.month)[1]
        data_fim = datetime(data_inicial.year, data_inicial.month, ultimo_dia)

        # Adiciona o intervalo à lista
        intervalos.append((data_inicial.strftime('%Y-%m-%d'), data_fim.strftime('%Y-%m-%d')))

        # Avança para o próximo mês
        if data_inicial.month == 12:
            data_inicial = datetime(data_inicial.year + 1, 1, 1)
        else:
            data_inicial = datetime(data_inicial.year, data_inicial.month + 1, 1)

    return intervalos

# Interface do usuário com campos de entrada que respeitam a lógica de validação de datas
codigo_unidade = '943001' # @param {type: "string"}
data_publicacao_inicial = '2023-01-01' # @param {type: "date"}
data_publicacao_final = '2023-12-31' # @param {type: "date"}
codigo_modalidade = 5 # @param {type: "number"}
pagina = 1 # @param {type: "number"}
tamanho_pagina = 200 # @param {type: "number"}

# Converter as datas para o formato datetime
data_inicial = datetime.strptime(data_publicacao_inicial, '%Y-%m-%d')
data_final = datetime.strptime(data_publicacao_final, '%Y-%m-%d')

# Verifica se a data inicial é menor que a data final
if data_inicial > data_final:
    raise ValueError("A data inicial não pode ser maior que a data final.")

# Gerar intervalos mensais
intervalos = gerar_intervalos(data_inicial, data_final)

# Coletar dados para cada intervalo
todos_dados = []
for inicio, fim in intervalos:
    print(f"Coletando dados de {inicio} a {fim}...")
    dados = coletar_dados(codigo_unidade, inicio, fim, codigo_modalidade, pagina, tamanho_pagina)
    if dados and dados['totalRegistros'] > 0:
        todos_dados.extend(dados['resultado'])

# Salvar todos os dados em uma planilha
if todos_dados:
    nome_arquivo = f"contratacoes_{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx"
    salvar_planilha({'resultado': todos_dados}, nome_arquivo)
else:
    print("Nenhum dado encontrado para o período especificado.")

from google.colab import files
files.download(nome_arquivo)


Coletando dados de 2023-01-01 a 2023-01-31...
Coletando dados de 2023-02-01 a 2023-02-28...
Coletando dados de 2023-03-01 a 2023-03-31...
Coletando dados de 2023-04-01 a 2023-04-30...
Coletando dados de 2023-05-01 a 2023-05-31...
Coletando dados de 2023-06-01 a 2023-06-30...
Coletando dados de 2023-07-01 a 2023-07-31...
Coletando dados de 2023-08-01 a 2023-08-31...
Coletando dados de 2023-09-01 a 2023-09-30...
Coletando dados de 2023-10-01 a 2023-10-31...
Coletando dados de 2023-11-01 a 2023-11-30...
Coletando dados de 2023-12-01 a 2023-12-31...
Planilha salva como contratacoes_20241003_170024.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>